considering the same churn dataset and problem statement but here the output feature is EstimatedSalary, so here will predict the salary

In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
df=pd.read_csv('../Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


for converting the gender and geography column to numeric data using the same pickle file created in classification, we can also create the pickle files again by following the same process

same scaler we can't use here as the independent and dependent features are changed here so need to create the new standardized scalar

In [20]:
with open('../classification/labelencoder.pkl','rb') as file:
    labelencoder=pickle.load(file)

with open('../classification/onehotencoder.pkl','rb') as file:
    onehotencoder=pickle.load(file)


In [6]:
df['Gender']=labelencoder.transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [15]:
geo=onehotencoder.transform(df[['Geography']]).toarray()
geo_df=pd.DataFrame(geo,columns=onehotencoder.get_feature_names_out(['Geography']))
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [16]:
df=pd.concat([df.drop('Geography',axis=1),geo_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
X=df.drop('EstimatedSalary',axis=1)
Y=df['EstimatedSalary']

In [18]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [21]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [38]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [44]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [45]:
model=Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)  # as its regression problem so by default linear activation function gets applied
])

c:\Users\srish\anaconda3\envs\langchain\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


make sure that after compiling the model you see the summary and use the model else it will through an error

In [46]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [48]:
log_dir='logs/fit/'+datetime.datetime.now().strftime("%Y%m%d=%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [49]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [50]:
# train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100781.2812 - mae: 100781.2812 - val_loss: 98516.7500 - val_mae: 98516.7500
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99505.4766 - mae: 99505.4766 - val_loss: 96975.8281 - val_mae: 96975.8281
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 97527.4141 - mae: 97527.4141 - val_loss: 93048.3438 - val_mae: 93048.3438
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 93387.1094 - mae: 93387.1094 - val_loss: 86489.0078 - val_mae: 86489.0078
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 87337.7109 - mae: 87337.7109 - val_loss: 77954.2344 - val_mae: 77954.2344
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 77462.2031 - mae: 77462.2031 - val_loss: 69175.7969 - val_mae: 69175.7969
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 68240.1328 - mae: 68240.1328 - val_loss: 61489.4102 - val_mae: 61489.4102
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step 

In [51]:
model.save('model_reg.h5')

In [52]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [55]:
# testing the model performance with testing data
test_loss, test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')
print(f'Test Loss: {test_loss}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51019.1445 - mae: 51019.1445
Test MAE: 50180.83203125
Test Loss: 50180.83203125
